In [39]:
import requests
import nltk
import spacy
from bs4 import BeautifulSoup
import urllib.parse
from PIL import Image
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor, ToPILImage, Resize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

nlp = spacy.load('en_core_web_sm')


import base64

def download_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        content_type = response.headers.get('Content-Type')
        if content_type and content_type.startswith('data:image'):
            image_data = response.content.split(b';base64,', 1)[1]
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(image_data))
        else:
            with open(filename, 'wb') as f:
                f.write(response.content)
        print(f"Downloaded {filename}")
        return True
    else:
        print(f"Failed to download {filename}")
        return False


def get_keywords(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)

    keywords = []
    for token, tag in tagged_tokens:
        if tag.startswith('NN'):  # Consider nouns as keywords
            keywords.append(token)

    return keywords


def get_base64_image_urls(sentence, num_images):
    keywords = get_keywords(sentence)
    if not keywords:
        return []

    search_query = ' '.join(keywords)
    search_query = urllib.parse.quote(search_query)
    search_url = f'https://www.google.com/search?q={search_query}&tbm=isch&hl=en&tbs=isz:l&sa=X&ved=0CAIQpwVqFwoTCPjenZDHz_8CFQAAAAAdAAAAABAE&biw=1519&bih=714'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(search_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        image_elements = soup.select('img[data-src]')

        base64_image_urls = []
        count = 0

        for image_element in image_elements:
            image_url = image_element['data-src']
            if image_url.startswith('data:image/jpeg;base64,'):
                base64_image_urls.append(image_url)
                count += 1
                if count >= num_images:
                    break

        return base64_image_urls

    return []


def download_images_collage(sentences, num_images=3, collage_size=(512, 512)):
    for i, sentence in enumerate(sentences):
        base64_image_urls = get_base64_image_urls(sentence, num_images)
        print(base64_image_urls)
        if base64_image_urls:
            collage_images = []
            for j, image_url in enumerate(base64_image_urls):
                filename = f"image_{i + 1}_{j + 1}.jpg"
                if download_image(image_url, filename):
                    image = Image.open(filename).convert("RGB")
                    image = Resize(collage_size)(image)
                    image_tensor = ToTensor()(image)
                    collage_images.append(image_tensor)

            collage = make_grid(collage_images, padding=10)
            collage = ToPILImage()(collage)
            collage.save(f"collage_{i + 1}.jpg")
            print(f"Saved collage for sentence {i + 1}")
        else:
            print(f"No images found for sentence {i + 1}")


# Example usage
sentences = [
    'Cricket is believed to have originated in the 16th century in England and is now played in over 100 countries worldwide.',
    'The longest recorded cricket match lasted for 10 days.',
    'Sachin Tendulkar is considered one of the greatest batsmen in the history of cricket.',
    'The Ashes is a famous cricket series played between England and Australia. It originated in 1882 and is one of the oldest rivalries in sports.'
]

download_images_collage(sentences, num_images=3)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[]
No images found for sentence 1
[]
No images found for sentence 2
[]
No images found for sentence 3
[]
No images found for sentence 4


In [52]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Set up your search query and number of images to fetch
search_query = 'lion'
num_images = 3

# Configure Selenium webdriver
driver = webdriver.Chrome()  # Make sure you have Chrome WebDriver installed and in the system PATH
driver.get("https://www.google.com/imghp")

# Find and fill the search bar
search_bar = driver.find_element(By.CSS_SELECTOR, "input[name='q']")
search_bar.send_keys(search_query)
search_bar.send_keys(Keys.RETURN)

# Wait for search results to load
time.sleep(5)

# Find image elements in the search results
image_elements = driver.find_elements(By.CSS_SELECTOR, "img[class='rg_i Q4LuWd']")

# Download the images
downloaded_images = 0
for i, image_element in enumerate(image_elements):
    # Click on the image to view it in a larger size
    driver.execute_script("arguments[0].click();", image_element)
    time.sleep(2)
    
    # Find the larger image element
    larger_image_element = driver.find_element(By.CSS_SELECTOR, "img[class='n3VNCb']")
    
    # Get the image source URL
    image_url = larger_image_element.get_attribute("src")
    if image_url.startswith('data:image'):
        continue  # Skip base64 encoded images
    
    try:
        # Send a GET request to download the image
        image_response = requests.get(image_url, stream=True)
        image_response.raise_for_status()
        
        # Save the image to a file
        image_path = f'downloaded_images/image_{downloaded_images + 1}.jpg'
        with open(image_path, 'wb') as image_file:
            shutil.copyfileobj(image_response.raw, image_file)
        
        print(f"Image {downloaded_images + 1} downloaded and saved as {image_path}")
        downloaded_images += 1
        
        if downloaded_images == num_images:
            break
    except Exception as e:
        print(f"Error downloading image: {str(e)}")

# Close the browser
driver.quit()

if downloaded_images == 0:
    print("No images found for the given search query.")


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 111
Current browser version is 114.0.5735.134 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe


In [24]:
import subprocess
from mutagen.mp3 import MP3
import os

def create_video(images, voiceovers, output_file):
    # Determine the maximum duration among all voiceovers
    max_duration = max([get_duration(vo) for vo in voiceovers])
    
    # Generate a temporary text file to store the list of image files
    image_list_file = 'image_list.txt'
    with open(image_list_file, 'w') as f:
        for image in images:
            f.write("file '{}'\n".format(image))
    
    # Generate a temporary text file to store the list of voiceover files
    voiceover_list_file = 'voiceover_list.txt'
    with open(voiceover_list_file, 'w') as f:
        for i, voiceover in enumerate(voiceovers):
            f.write("file '{}'\n".format(voiceover))
            f.write("duration {}\n".format(max_duration))
    
    # Execute the ffmpeg command to create the video
    subprocess.call([
        'ffmpeg', '-y',
        '-f', 'concat',
        '-safe', '0',
        '-i', image_list_file,
        '-f', 'concat',
        '-safe', '0',
        '-i', voiceover_list_file,
        '-c:v', 'libx264',
        '-vf', 'fps=30',
        '-pix_fmt', 'yuv420p',
        output_file
    ])
    
    # Remove the temporary text files
    os.remove(image_list_file)
    os.remove(voiceover_list_file)
    
def get_duration(voiceover_file):
    audio = MP3(voiceover_file)
    duration = audio.info.length
    return duration

# Example usage
images = ['image_1.jpg', 'image_2.jpg', 'image_3.jpg', 'image_4.jpg', 'image_5.jpg', 'image_6.jpg']
voiceovers = ['voiceover_1.mp3', 'voiceover_2.mp3', 'voiceover_3.mp3', 'voiceover_4.mp3', 'voiceover_5.mp3', 'voiceover_6.mp3']
output_file = 'output.mp4'

create_video(images, voiceovers, output_file)


HeaderNotFoundError: can't sync to MPEG frame

In [27]:
import plotly.figure_factory as ff

# Define module names
modules = ['Module 1', 'Module 2', 'Module 3', 'Module 4', 'Module 5', 'Module 6']

# Define start and end dates for each module (in weeks)
start_dates = ['2023-06-26', '2023-07-04', '2023-07-12', '2023-07-20', '2023-07-28', '2023-08-5']
end_dates = ['2023-07-03', '2023-07-11', '2023-07-19', '2023-07-27', '2023-08-4', '2023-08-10']

# Define costs in USD
costs = [270, 450, 540, 359, 180, None]

# Define requirements and other necessary information
requirements = [
'Identity Logo and UI/UX design for Web, App, and Admin Panel',
'Workable Authentication for Admin Panel, Website, and App along with Basic Front-end',
'Complete Seller and Buyer Implementation in App and Web, and Stats Related to Seller/Buyer in Admin Panel',
'Complete Web, App, and Admin Panel',
'Testing',
'Deployment'
]

# Create a list of dictionaries with the data
data = [
{'Task': module, 'Start': start, 'Finish': end, 'Cost': cost, 'Requirements': requirement}
for module, start, end, cost, requirement in zip(modules, start_dates, end_dates, costs, requirements)
]

# Create a Gantt chart figure
fig = ff.create_gantt(
data,
colors=['rgb(158,202,225)', 'rgb(58,200,225)', 'rgb(58,160,255)', 'rgb(58,100,255)', 'rgb(58,0,255)', 'rgb(0,0,0)'],
index_col='Requirements',
title='Project Timeline',
showgrid_x=True,
showgrid_y=True,
bar_width=0.3,
group_tasks=True
)

# Update axis labels
fig.update_layout(
xaxis=dict(title='Weeks'),
yaxis=dict(title='Modules')
)

# Update hovertemplate to show details in the legend
fig.update_traces(hovertemplate='<b>%{text}</b><br>Start: %{x}<br>End: %{y}<br>Cost: $%{customdata[0]}')

# Add cost annotation on the bars
for module, end, cost in zip(modules, end_dates, costs):
    fig.add_annotation(
    x=end,
    y=module,
    text=f"${cost}",
    font=dict(color='black', size=12),
    showarrow=False,
    align='center',
    xshift=5
    )

# Show the Gantt chart
fig.show()